In [1]:
import pandas as pd
import logging
import time
import requests
import groq
from groq import Groq
import re

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")

# File paths and table configuration
input_excel_file = r"C:\Users\leeja\Source\Repos\NewKinsTake4\NewKinsTake4\wwwroot\data\jakePython\test.xlsx"
sheet_name = "workflow_QL_CT"
column_to_translate = "WorkStageName"
translated_column = "WorkStageNameKo"

# API client setup
client = Groq(api_key="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", max_retries=3)
model_type = "llama3-groq-70b-8192-tool-use-preview"

# System prompt for the translation model
system_prompt = (
    "You are a translation assistant specializing in the finance sector, "
    "familiar with banking system screen functions and workflows. The text you are translating "
    "contains financial jargon and UI labels commonly used in banking software interfaces. "
    "Provide accurate translations in Korean while preserving the context and meaning of the original text."
    "Keep in mind that <br> is there for the purpose of 'Mark down', therefore leave it at its current location."
)

# Function to check if text contains Korean characters
def contains_korean(text):
    return bool(re.search(r'[\uac00-\ud7a3]', str(text)))  # Check for Korean Unicode range

# Translation function using Groq API
def translate_to_korean(text):
    text = text.replace("*", "")
    
    if not text or pd.isna(text) or len(text.strip()) < 5:
        return text

    retries = 3
    for attempt in range(retries):
        try:
            time.sleep(3)
            response = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"Translate the following text to Korean: '{text}'"}
                ],
                model=model_type,
                temperature=0.2,
                max_tokens=500,
                top_p=1,
                stream=False,
                stop="[end_of_sent]",
                seed=100,
            )
            translated_text = response.choices[0].message.content.strip()
            logging.info(f"Translated: {translated_text}")
            return translated_text
        except groq.RateLimitError:
            logging.warning(f"Rate limit error on attempt {attempt + 1}. Retrying...")
            time.sleep(5)
        except groq.APIConnectionError as e:
            logging.error(f"API connection error: {e}")
            break
        except groq.APIStatusError as e:
            logging.error(f"API status error: {e.response}")
            break

    logging.error("Translation failed after multiple retries.")
    return text

# Main function to process the Excel file
def translate_excel_columns():
    try:
        logging.info("Loading data from the Excel file...")
        df = pd.read_excel(input_excel_file, sheet_name=sheet_name)

        # Add the translated column if it doesn't exist
        if translated_column not in df.columns:
            logging.info(f"Adding column: {translated_column}")
            df[translated_column] = None

        # Apply translation only if 'LoadingPageType' is 'Image', and no Korean text is present
        logging.info(f"Translating column: {column_to_translate}")
        df[translated_column] = df.apply(
            lambda row: translate_to_korean(row[column_to_translate]) 
            if row['LoadingPageType'] == 'Image' and 
               pd.notna(row[column_to_translate]) and 
               len(str(row[column_to_translate]).strip()) >= 5 and 
               not contains_korean(row[translated_column])
            else row[translated_column],
            axis=1
        )

        # Save the translated data back to the Excel file
        logging.info("Saving the translated data to the Excel file...")
        backup_file = input_excel_file.replace(".xlsx", "_backup.xlsx")
        df.to_excel(backup_file, sheet_name=sheet_name, index=False)
        df.to_excel(input_excel_file, sheet_name=sheet_name, index=False)
        logging.info("Translation process completed successfully.")

    except FileNotFoundError:
        logging.error(f"File not found: {input_excel_file}")
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")

# Execute the script
if __name__ == "__main__":
    translate_excel_columns()


2024-11-28 13:19:41,404 - Loading data from the Excel file...
2024-11-28 13:19:41,699 - Translating column: WorkStageName
2024-11-28 13:19:45,807 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-28 13:19:45,818 - Translated: 이 화면의 기능은 고객이 제공하는 담보의详细한 정보를 등록하는 것입니다.  <br>  <br>  ① 시스템은 부동산 (기계), 동산, 예금, 신용장, 증권 등 다양한 유형의 담보를 지원합니다.  <br>  ② 각 담보는 개별적으로 등록되어야 합니다.  <br>  ③ 예금 기반 담보의 경우, 예금이 은행 내에 있으면 예금의 위임 및 해제 프로세스가 자동으로 관리됩니다.  <br>  <br>  이 화면은 준수 및 운영 요구 사항을 위해 고객이 제공한 담보 데이터를 정확하고 효율적으로 관리하는 것을 보장합니다.
2024-11-28 13:19:49,626 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-28 13:19:49,641 - Translated: 이 화면의 기능은 등록된 담보 기록을 확인하고 대출에 연결하는 것입니다. <br> <br> 사용자는 다음을 수행할 수 있습니다. <br> ① 재평가 방법을 분류하고 검토하는 것, <br> ② 담보 평가 방법을 선택하는 것, <br> ③ 우선순위 및 관련 세부 사항을 확인하고 확인하는 것. <br> <br> 이 화면은 대출-담보 연결 프로세스를 포괄적으로 관리하여 대출-담보 연결을 정확하고 체계적으로 처리합니다.
2024-11-28 13:19:53,477 - HTTP Request: POST https://api.groq.

KeyboardInterrupt: 